<a href="https://colab.research.google.com/github/bhupesh-varma/PhysiSolve/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step - 1
### spplitting the dataset into training, testing and evaluation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import random
from sklearn.model_selection import train_test_split
from collections import Counter

# Set random seed for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

# Load the dataset
with open(r"/content/drive/MyDrive/dataset/high_school_physics.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Extract the 'subject' field for stratification
subjects = [item["subject"] for item in data]

# Verify initial distribution
print("Original subject distribution:", Counter(subjects))

# First split: 70% train, 30% temp (test + eval)
train_data, temp_data = train_test_split(
    data,
    train_size=0.7,
    stratify=subjects,
    random_state=RANDOM_SEED
)

# Second split: Split the 30% temp into 15% test and 15% eval (50/50 of temp)
test_data, eval_data = train_test_split(
    temp_data,
    test_size=0.5,  # 50% of 30% = 15% of original
    stratify=[item["subject"] for item in temp_data],
    random_state=RANDOM_SEED
)

# Save train, test, and evaluation sets
with open(r"/content/drive/MyDrive/dataset/train.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, indent=4)
with open(r"/content/drive/MyDrive/dataset/test.json", "w", encoding="utf-8") as f:
    json.dump(test_data, f, indent=4)
with open(r"/content/drive/MyDrive/dataset/eval.json", "w", encoding="utf-8") as f:
    json.dump(eval_data, f, indent=4)

# Print split sizes and subject distribution
print(f"\nDataset split into {len(train_data)} training, {len(test_data)} testing, and {len(eval_data)} evaluation samples.")
print("\nSubject distribution in each split:")
print("Train:", Counter([item["subject"] for item in train_data]))
print("Test:", Counter([item["subject"] for item in test_data]))
print("Eval:", Counter([item["subject"] for item in eval_data]))

Original subject distribution: Counter({'Electrostatics and Current Electricity': 76, 'Mechanics': 60, 'Kinematics': 55, 'Electromagnetism': 45, 'Thermodynamics': 44, 'Optics': 38, 'Atomic and Modern Physics': 30, 'Electronic Devices': 29, 'Periodic Motion': 13, 'Waves and Oscillations': 10})

Dataset split into 280 training, 60 testing, and 60 evaluation samples.

Subject distribution in each split:
Train: Counter({'Electrostatics and Current Electricity': 53, 'Mechanics': 42, 'Kinematics': 39, 'Thermodynamics': 31, 'Electromagnetism': 31, 'Optics': 27, 'Atomic and Modern Physics': 21, 'Electronic Devices': 20, 'Periodic Motion': 9, 'Waves and Oscillations': 7})
Test: Counter({'Electrostatics and Current Electricity': 11, 'Mechanics': 9, 'Kinematics': 8, 'Electromagnetism': 7, 'Thermodynamics': 6, 'Optics': 5, 'Atomic and Modern Physics': 5, 'Electronic Devices': 5, 'Periodic Motion': 2, 'Waves and Oscillations': 2})
Eval: Counter({'Electrostatics and Current Electricity': 12, 'Mechan

### Analysis of the Output
#### Original Distribution
- **Total samples**: 400
- **Subjects**:
  - Electrostatics and Current Electricity: 76
  - Mechanics: 60
  - Kinematics: 55
  - Electromagnetism: 45
  - Thermodynamics: 44
  - Optics: 38
  - Atomic and Modern Physics: 30
  - Electronic Devices: 29
  - Periodic Motion: 13
  - Waves and Oscillations: 10

#### Split Results
- **Train**: 280 samples (70%)
- **Test**: 60 samples (15%)
- **Eval**: 60 samples (15%)

#### Subject Distribution Across Splits
| Subject                          | Original | Train (70%) | Test (15%) | Eval (15%) |
|----------------------------------|----------|-------------|------------|------------|
| Electrostatics and Current Elec. | 76       | 53 (53.2)   | 11 (11.4)  | 12 (11.4)  |
| Mechanics                        | 60       | 42 (42)     | 9 (9)      | 9 (9)      |
| Kinematics                       | 55       | 39 (38.5)   | 8 (8.25)   | 8 (8.25)   |
| Electromagnetism                 | 45       | 31 (31.5)   | 7 (6.75)   | 7 (6.75)   |
| Thermodynamics                   | 44       | 31 (30.8)   | 6 (6.6)    | 7 (6.6)    |
| Optics                           | 38       | 27 (26.6)   | 5 (5.7)    | 6 (5.7)    |
| Atomic and Modern Physics        | 30       | 21 (21)     | 5 (4.5)    | 4 (4.5)    |
| Electronic Devices               | 29       | 20 (20.3)   | 5 (4.35)   | 4 (4.35)   |
| Periodic Motion                  | 13       | 9 (9.1)     | 2 (1.95)   | 2 (1.95)   |
| Waves and Oscillations           | 10       | 7 (7)       | 2 (1.5)    | 1 (1.5)    |


### Few-Shot Evaluation of Flan-T5 from Hugging Face

In [4]:
from transformers import pipeline

qa_pipeline = pipeline(
    "text-generation",
    model="microsoft/phi-3-mini-4k-instruct",
    device_map = "auto",
    torch_dtype="auto"
)

def evaluate_model(model, dataset):
    correct = 0
    total = len(dataset)

    for item in dataset:
        question = item["question"]
        idx = ord(item["answer"])-ord("A")
        correct_answer = item["choices"][idx]

        prompt = f"Give me the final answer without any explaination, just the couple of words with units that directly show the answer for the Question: {question} with Choices: {', '.join(item['choices'])} Answer:"
        prediction = model(prompt, max_length=100, truncation=True)[0]["generated_text"]

        predicted_answer = prediction.split("Answer:")[-1].strip()
        if correct_answer.lower() in prediction.lower():
          correct += 1

    accuracy = (correct / total) * 100
    return accuracy

zero_shot_accuracy = evaluate_model(qa_pipeline, test_data)
print(f"Zero-Shot Accuracy: {zero_shot_accuracy:.2f}%")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


Zero-Shot Accuracy: 100.00%


### Fine-Tuning Flan-T5 using LoRA

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
import json
import os
import time

# Disable W&B logging
os.environ["WANDB_DISABLED"] = "true"

# Load Phi-3 model and tokenizer
model_name = "microsoft/phi-3-mini-128k-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float32)  # Use float32 for CPU
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create custom dataset
class PhysicsDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        prompt = f"Question: {item['question']}\nChoices: {', '.join(item['choices'])}\nAnswer:"
        idx = ord(item["answer"][0]) - ord("A")
        target = item["choices"][idx]

        # For causal models, concatenate input and target as one string
        full_text = prompt + " " + target
        encoding = self.tokenizer(full_text, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")

        input_ids = encoding["input_ids"].squeeze()
        attention_mask = encoding["attention_mask"].squeeze()

        # Labels are the same as input_ids, but mask out padding tokens
        labels = input_ids.clone()
        labels[attention_mask == 0] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

# Load data
train_file_path = "/content/drive/MyDrive/dataset/train.json"
eval_file_path = "/content/drive/MyDrive/dataset/eval.json"

with open(train_file_path, "r", encoding="utf-8") as f:
    train_data = json.load(f)

with open(eval_file_path, "r", encoding="utf-8") as f:
    eval_data = json.load(f)

train_dataset = PhysicsDataset(train_data, tokenizer)
eval_dataset = PhysicsDataset(eval_data, tokenizer)

# Output directories
model_save_dir = "/content/drive/MyDrive/dataset/trained_model_phi3"
results_dir = "/content/drive/MyDrive/dataset/results_phi3"
os.makedirs(model_save_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

# Training arguments
training_args = TrainingArguments(
    output_dir=results_dir,
    per_device_train_batch_size=2,  # Smaller batch size for Phi-3
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    run_name=f"phi3-mini-{time.strftime('%Y%m%d-%H%M%S')}",
    report_to="none",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train
trainer.train()

# Save model
model.save_pretrained(model_save_dir)
tokenizer.save_pretrained(model_save_dir)
print(f"Phi-3 model and tokenizer saved to {model_save_dir}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Testing the trained model on test.json

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import json
import torch
import os
from tqdm import tqdm

# Paths
model_save_dir = "/content/drive/MyDrive/dataset/trained_model"
test_file_path = "/content/drive/MyDrive/dataset/test.json"  # Adjust if your test file has a different name

# First, let's check if the model files exist
print("Checking model directory contents:")
for root, dirs, files in os.walk(model_save_dir):
    for file in files:
        print(os.path.join(root, file))

# Load the model and tokenizer
try:
    model = T5ForConditionalGeneration.from_pretrained(model_save_dir)
    tokenizer = T5Tokenizer.from_pretrained(model_save_dir)
    print("Model and tokenizer loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    # If model loading fails, load the original pretrained model
    print("Loading the base model instead...")
    model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")
    tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large", legacy=False)

# Load test data
with open(test_file_path, "r", encoding="utf-8") as f:
    test_data = json.load(f)

print(f"Loaded {len(test_data)} test examples")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Testing function
def evaluate_model(model, tokenizer, test_data, device, correct, total):

    for item in tqdm(test_data):
        prompt = f"Question: {item['question']} Choices: {', '.join(item['choices'])} Answer:"
        correct_idx = ord(item["answer"][0]) - ord("A")
        correct_answer = item["choices"][correct_idx]

        # Tokenize input
        input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=128).input_ids.to(device)

        # Generate output
        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                max_length=128,
                num_beams=4,
                early_stopping=True
            )

        # Decode output
        predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Simple string match for evaluation
        if predicted_text.strip() == correct_answer.strip():
            correct += 1
        else:
            # Print some examples of wrong predictions for debugging
            if total < 5:  # Limit to just a few examples
                print(f"\nQuestion: {item['question']}")
                print(f"Choices: {', '.join(item['choices'])}")
                print(f"Correct Answer: {correct_answer}")
                print(f"Predicted: {predicted_text}")

        total += 1

    accuracy = correct / total if total > 0 else 0
    return accuracy

# Evaluate model
print("Evaluating model on test set...")
correct = 0
total = 0
accuracy = evaluate_model(model, tokenizer, test_data, device, correct, total)
print(f"Test Accuracy: {accuracy:.4f} ({correct}/{total})")

Checking model directory contents:
Error loading model: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory /content/drive/MyDrive/dataset/trained_model.
Loading the base model instead...
Loaded 60 test examples
Evaluating model on test set...


  2%|▏         | 1/60 [00:05<05:38,  5.74s/it]


Question: A 10 Ω resistor is connected across a 15 V battery. What is the current flowing through the resistor?
Choices: 0.5 A, 1 A, 1.5 A, 2 A
Correct Answer: 1.5 A
Predicted: 2 A


  3%|▎         | 2/60 [00:11<05:44,  5.94s/it]


Question: A force of 100 N is applied to a 50 kg object. What is the acceleration of the object?
Choices: 1 m/s², 2 m/s², 3 m/s², 4 m/s²
Correct Answer: 2 m/s²
Predicted: 4 m/s2


...


Question: A force of 80 N is applied to a 8 kg block. What is the acceleration of the block?
Choices: 5 m/s², 10 m/s², 15 m/s², 20 m/s²
Correct Answer: 10 m/s²
Predicted: 20 m/s2


  8%|▊         | 5/60 [00:21<03:35,  3.92s/it]


Question: Two resistors of 15Ω and 5Ω are connected in parallel. What is the equivalent resistance?
Choices: 1Ω, 2Ω, 3.75Ω, 5Ω
Correct Answer: 3.75Ω
Predicted: 5


100%|██████████| 60/60 [03:16<00:00,  3.27s/it]

Test Accuracy: 0.2667 (0/0)


In [ ]:
import torch
import json
from transformers import T5ForConditionalGeneration, T5Tokenizer
from torch.utils.data import Dataset
import pickle

# Load trained model
model_path = "/content/drive/MyDrive/dataset/trained_model.pkl"
with open(model_path, "rb") as f:
    model = pickle.load(f)

# Load tokenizer
model_name = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)

# Load test dataset
test_file_path = "/content/drive/MyDrive/dataset/test.json"
with open(test_file_path, "r", encoding="utf-8") as f:
    test_data = json.load(f)

# Define test dataset class
class PhysicsDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        prompt = f"Question: {item['question']} Choices: {', '.join(item['choices'])} Answer:"
        idx = ord(item["answer"][0])-ord("A")  # Convert letter to index
        target = item["choices"][idx]

        encodings = self.tokenizer(prompt, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")
        target_encodings = self.tokenizer(target, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")

        return {
            "input_ids": encodings["input_ids"].squeeze(),
            "attention_mask": encodings["attention_mask"].squeeze(),
            "labels": target_encodings["input_ids"].squeeze(),
            "correct_answer": target
        }

# Prepare test dataset
test_dataset = PhysicsDataset(test_data, tokenizer)

# Function to evaluate accuracy
def evaluate_accuracy(model, dataset, tokenizer):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    for item in dataset:
        input_ids = item["input_ids"].unsqueeze(0).to("cuda" if torch.cuda.is_available() else "cpu")
        attention_mask = item["attention_mask"].unsqueeze(0).to("cuda" if torch.cuda.is_available() else "cpu")

        # Generate prediction
        with torch.no_grad():
            output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128)

        # Decode predicted and actual answer
        predicted_answer = tokenizer.decode(output[0], skip_special_tokens=True)
        correct_answer = item["correct_answer"]

        # Compare answers
        if predicted_answer.strip().lower() == correct_answer.strip().lower():
            correct += 1
        total += 1

    accuracy = (correct / total) * 100
    return accuracy

# Run evaluation
accuracy = evaluate_accuracy(model, test_dataset, tokenizer)
print(f"Test Accuracy: {accuracy:.2f}%")


EOFError: Ran out of input